# ID2214/FID3214 Assignment 2 Group no. 12
### Project members: 
[Enter Name, email]
[Enter Name, email]
[Enter Name, email]
[Enter Name, email]

### Declaration:
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas and time

In [60]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

## Reused functions from Assignment 1

In [90]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

def create_column_filter(df):
    
    # Copies the input df
    dfCopy = df.copy()  
   
    # Drops each column with only one unique value + one unique value with nan(s)
    for col in dfCopy.columns:
        if (col != "CLASS" and col != "ID") and (len(dfCopy[col].unique()) == 1 or (len(dfCopy[col].unique()) == 2 and dfCopy.isnull().values.any())):
            dfCopy.drop(col,inplace=True,axis=1)
             
    # Extracting column names and converts to list
    remainingColumns = dfCopy.columns.values.tolist()
    print("HEEJ")
    return dfCopy, remainingColumns

def apply_column_filter(df, column_filter):
    
    # Copies the input df
    dfCopy = df.copy()
    
    # Finds all similar column names and keep the values of them
    # Discards the rest
    df_final = dfCopy[dfCopy.columns.intersection(column_filter)]
    return df_final

def create_normalization(df, normalizationtype = "minmax"):
  
    # Creates df copy
    dfCopy = df.copy()
    
    # Empty dict for normalization
    normalization = {}

    # Iterating through the columns
    for col in dfCopy.columns:
        
        # Checks type and column name
        if (col != "CLASS" and col != "ID") and (dfCopy[col].dtype == np.int64 or dfCopy[col].dtype == np.float64):
            
            # Fetches min and max for each column and applies normalization
            # Adds to dictionary
            if(normalizationtype == "minmax"):
                min = dfCopy[col].min()
                max = dfCopy[col].max()
                dfCopy[col] = [(x-min)/(max-min) for x in dfCopy[col]]
                normalization[col] = ("minmax", min, max)
                
            # Calculates mean and standard deviation for each column and applies noralization
            # Adds to dictionary
            elif (normalizationtype == "zscore"):
                mean = dfCopy[col].mean()
                std = dfCopy[col].std()
                dfCopy[col] = df[col].apply(lambda x: (x-mean)/std)
                normalization[col] = ("zscore",mean,std)

    return dfCopy, normalization

def apply_normalization(df, normalization):
    
    # Creates df copy
    dfCopy = df.copy()
    
    # Applies different normalization based on type for each column 
    for col in dfCopy:
        if(col != "CLASS" and col != "ID"):
            if(normalization[col][0] == 'minmax'):
                dfCopy[col] = [(x-normalization[col][1])/(normalization[col][2]-normalization[col][1]) for x in dfCopy[col]]
            elif(normalization[col][0] == 'zscore'):
                dfCopy[col].apply(lambda x: (x- normalization[col][1])/normalization[col][2])
            
    return dfCopy    

def create_imputation(df):
    dfCopy = df.copy()
    dictionary = {}
    
    for col in dfCopy:
        
        # Checks type and column name
        if (col != "CLASS" and col != "ID") and (dfCopy[col].dtype == np.int64 or dfCopy[col].dtype == np.float64):
                if dfCopy[col].isnull().all():
                    dfCopy[col].fillna(0, inplace=True)
                    dictionary[col] = 0 
                else:
                    mean = dfCopy[col].mean()
                    dfCopy[col].fillna(mean, inplace=True)
                    dictionary[col] = mean 
        elif dfCopy[col].dtype == np.object:
                if dfCopy[col].isnull().all():
                    dfCopy[col].fillna('', inplace=True)
                    dictionary[col] = '' 
                else:
                    mode = dfCopy[col].mode()
                    dfCopy[col].fillna(mode[0], inplace=True)
                    dictionary[col] = mode[0] 
                    
    return dfCopy, dictionary

def apply_imputation(df, dictionary):
    dfCopy = df.copy()
    
    for col in dictionary:
        dfCopy.fillna(dictionary[col], inplace=True)
        
    return dfCopy


def create_one_hot(df):
    dfCopy = df.copy()
    one_hot = {}

    for col in dfCopy.columns:
        if (col != "CLASS" and col != "ID"):

            dfCopy[col] = dfCopy[col].astype("category")
            one_hot[col]= list(dfCopy[col].cat.categories)

            y = pd.get_dummies(dfCopy[col], prefix = col, dtype=float)
            dfCopy = pd.concat([dfCopy, y], axis=1)

    dfCopy.drop(dfCopy.iloc[:, :9], inplace = True, axis = 1)

    return dfCopy, one_hot

def apply_one_hot(df, one_hot):
    dfCopy = df.copy()
    
    for col in one_hot:
        y = pd.get_dummies(dfCopy[col], dtype=float)
        for cat in one_hot[col]:
            dfCopy[col+'-'+cat] = y[cat]
    
    dfCopy.drop(dfCopy.iloc[:, :9], inplace = True, axis = 1)

    return dfCopy

def accuracy(df, correctlabels):
    dfCopy = df.copy()
    
    highestLabels = dfCopy.idxmax(axis=1)
    highestLabelsList = highestLabels.values.tolist()
    
    count = 0
    dfLength = len(dfCopy["A"])
    
    for i in range(len(correctlabels)):
        if correctlabels[i] == highestLabelsList[i]:
            count = count + 1
    
    res = count / dfLength
    
    return res

def brier_score(df, correctlabels):
    dfCopy = df.copy()
    
    correctDf = pd.get_dummies(correctlabels)
    
    brier_score = np.mean(np.sum((dfCopy-correctDf)**2, axis= 1))
    
    return brier_score

def get_pos_and_neg(predictions, pos_label, correct_labels):
    scores = {}
    
    # Counts tps amd fps and put them in a dict
    for i in range(len(predictions)):
        score = predictions[pos_label][i] 
        (p,n) = scores.get(score,(0,0))
        if correct_labels[i] == pos_label:
            scores[score] = (p+1,n)
        else:
            scores[score] = (p,n+1)

    # dict to array
    # sort array
    scores = np.array([[score,scores[score][0],scores[score][1]] for score in scores.keys()])
    scores = scores[np.argsort(scores[:,0])[::-1]]
    
    # drop predictions
    pos = scores[:,1]
    neg = scores[:,2]
    
    return pos, neg


def auc(predictions, correctlabels):
    
    import matplotlib.pyplot as plt
    
    AUC = 0
    
    for col in predictions.columns:
        
        #Räknar ut antal labels i correct labels
        class_frequency = correctlabels.count(col)
        
        # Gets total true positives and true negatives for each column
        pos, neg = get_pos_and_neg(predictions,col, correctlabels)
        
        # Area for each column
        thisAUC = calculateAUC(neg, pos, 0)
        
        # Total area for the predictions
        # "How many labels (e.g. A) do we have in correct labels (in 
        # This step considers ALL classes and calculates tot AUC, considering each column
        AUC += thisAUC*(class_frequency/len(correctlabels))
   
        # Ployt
        tpr = [cs/sum(pos) for cs in np.cumsum(pos)]
        fpr = [cs/sum(neg) for cs in np.cumsum(neg)]

        plt.plot([0.0]+fpr+[1.0],[0.0]+tpr+[1.0],"-",label=col)
        plt.plot([0.0,1.0],[0.0,1.0],"--",label="Baseline")
        plt.xlabel("fpr")
        plt.ylabel("tpr")
        plt.legend()
        plt.show()

    return AUC
    
    
def calculateAUC(neg, pos, AUC):
    Tot_tp = sum(pos)
    Tot_fp = sum(neg)
    
    Cov_tp = 0
    for i in range(len(pos)):
        
        # Cov tp holds true positives
        AUC += (Cov_tp/Tot_tp)*(neg[i]/Tot_fp)+ (pos[i]/Tot_tp)*(neg[i]/Tot_fp)/2
        Cov_tp += pos[i]

    return AUC



## 1. Define the class kNN

In [109]:
# Define the class kNN with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, normalization, one_hot, labels, training_labels, training_data, training_time
#
# Input to fit:
# self              - the object itself
# df                - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype - "minmax" (default) or "zscore"
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter   - a column filter (see Assignment 1) from df
# self.imputation      - an imputation mapping (see Assignment 1) from df
# self.normalization   - a normalization mapping (see Assignment 1), using normalizationtype from the imputed df
# self.one_hot         - a one-hot mapping (see Assignment 1)
# self.training_labels - a pandas series corresponding to the "CLASS" column, set to be of type "category" 
# self.labels          - a list of the categories (class labels) of the previous series
# self.training_data   - the values (an ndarray) of the transformed dataframe, i.e., after employing imputation, 
#                        normalization, and possibly one-hot encoding, and also after removing the "CLASS" and "ID" columns
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# k    - an integer >= 1 (default = 5)
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the relative class frequencies in the set of class labels from the k nearest 
#               (with respect to Euclidean distance) neighbors in training_data
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply column filtering, imputation, normalization and one-hot
#
# Hint 2: Get the numerical values (as an ndarray) from the resulting dataframe and iterate over the rows 
#         calling some sub-function, e.g., get_nearest_neighbor_predictions(x_test,k), which for a test row
#         (numerical input feature values) finds the k nearest neighbors and calculate the class probabilities.
#
# Hint 3: This sub-function may first find the distances to all training instances, e.g., pairs consisting of
#         training instance index and distance, and then sort them according to distance, and then (using the indexes
#         of the k closest instances) find the corresponding labels and calculate the relative class frequencies

class kNN:

    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.normalization = None
        self.one_hot = None
        self.labels = None 
        self.training_labels = None 
        self.training_data = None 
        self.training_time = None

    def fit(self, df, normalizationtype = "minmax"):
        
        df, self.column_filter = create_column_filter(df)
        df, self.imputation = create_imputation(df)
        df, self.normalization = create_normalization(df, normalizationtype)
        display("train df: ", df)
        #df, self.one_hot = create_one_hot(df)
       # display("one hot: ", self.one_hot)
        self.training_labels = df["CLASS"].astype("category")
        self.labels = pd.value_counts(df.CLASS).index.tolist() #df.groupby(by=self.training_labels)['CLASS'].nunique().sort_values(ascending=False).unique().tolist()
        self.training_data = df.drop(columns = ["CLASS", "ID"]).to_numpy()
        
        print("train values: ", self.training_data)
    #    plt.scatter(self.traning_data, c=red);    
     #   plt.show();
        
    def predict(self, df, k = 5):
        
        dfCopy = df.copy();
        
        
        dfCopy.drop(columns = ["CLASS", "ID"], inplace=True)
        dfCopy = apply_column_filter(dfCopy, self.column_filter)
        dfCopy = apply_imputation(dfCopy, self.imputation)
        dfCopy = apply_normalization(dfCopy, self.normalization)
        testValues = dfCopy.to_numpy()
        
        
        display("test values: ", testValues)
        #values = fit(self, dfCopy, normalizationtype="minmax")
        
        self.get_nearest_neighbor_predictions(testValues, k)
        
        
        #print(values)
    def get_nearest_neighbor_predictions(self, x_test,k):
        
        #eucl_distance = [eucledean_distance(data, tr_data_utan_klass) for data in okanda_objekt]; 
        #För varje datapunkt i datasettet med okända objekt beräknas euklidiska distansen till intilliggande (K) punkter. Detta för att avgöra klasstillhörighet.

        
        argSortedDistance = np.sort([self.eucledean_distance(x_test,x_train) for x_train in self.training_data], axis=0)
        
        nearestNeighbors = argSortedDistance[:k]
        print(nearestNeighbors)
    
    # 1. calculate the distance from our data points in test to data points in train
    # 2. then we want to check the k nearest neighbours for each data point in test
    # 3. get the class labels of the neighbour datapoints
    # 4a. compare the class labels (e.g. if class labels are (2,1,6,7,2), our test datapoint belongs to class 2)
    # 4b. (2,2,1,1,2) BUT every 2 is 10 units away from test data point x, and every 1 is 0.1 units away from test data point x
    # 4c. also, add weight when classifying data point x 
    
    def eucledean_distance(self, x, y):
        return np.sqrt(np.sum((x-y)**2, axis=0));
        
        
        

In [110]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

knn_model = kNN()

t0 = time.perf_counter()
knn_model.fit(glass_train_df)
print(kNN)

print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

test_labels = glass_test_df["CLASS"]
print(kNN)

k_values = [1,3,5,7,9]
results = np.empty((len(k_values),3))

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = knn_model.predict(glass_test_df,k=k_values[i])
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

HEEJ


'train df: '

,ID,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,CLASS
0,202,0.261785,0.241107,0.000000,0.327273,1.000000,0.434783,0.378378,0.000000,0.000000,7
1,124,0.288867,0.543478,0.775056,0.516364,0.504655,0.099839,0.276757,0.000000,0.000000,2
2,152,0.499498,0.709486,0.868597,0.196364,0.314711,0.000000,0.438919,0.000000,0.000000,3
3,197,0.213139,0.620553,0.000000,0.818182,0.636872,0.022544,0.430270,0.257143,0.027027,7
4,144,0.289870,0.448617,0.772829,0.545455,0.541899,0.106280,0.297297,0.000000,0.000000,2
...,...,...,...,...,...,...,...,...,...,...,...
102,178,0.404213,0.604743,0.536748,0.327273,0.549348,0.000000,0.469189,0.000000,0.000000,6
103,160,0.333501,0.547431,0.748330,0.487273,0.396648,0.091787,0.365405,0.000000,0.243243,3
104,88,0.257773,0.527668,0.777283,0.447273,0.528864,0.107890,0.286486,0.000000,0.270270,2
105,98,0.306921,0.290514,0.723831,0.316364,0.696462,0.099839,0.375135,0.000000,0.648649,2


train values:  [[0.26178536 0.24110672 0.         0.32727273 1.         0.43478261
  0.37837838 0.         0.        ]
 [0.2888666  0.54347826 0.77505568 0.51636364 0.50465549 0.09983897
  0.27675676 0.         0.        ]
 [0.4994985  0.70948617 0.86859688 0.19636364 0.31471136 0.
  0.43891892 0.         0.        ]
 [0.21313942 0.62055336 0.         0.81818182 0.63687151 0.02254428
  0.43027027 0.25714286 0.02702703]
 [0.28986961 0.4486166  0.77282851 0.54545455 0.54189944 0.10628019
  0.2972973  0.         0.        ]
 [0.27983952 0.38339921 0.64142539 0.51636364 0.63314711 0.11755233
  0.33621622 0.         0.        ]
 [0.25777332 0.83201581 0.         0.57454545 0.61452514 0.
  0.35027027 0.43809524 0.        ]
 [0.21865597 0.50592885 0.76837416 0.33454545 0.54562384 0.09017713
  0.33945946 0.         0.        ]
 [0.38465396 0.56324111 0.74610245 0.34181818 0.42271881 0.09500805
  0.37621622 0.         0.        ]
 [0.35005015 0.49604743 0.86859688 0.40727273 0.46927374 0.088566

'test values: '

array([[ 0.26278837,  0.39920949,  0.63474388,  0.41818182,  0.6443203 ,
         0.09178744,  0.36324324,  0.03492063,  0.59459459],
       [ 0.79939819,  0.60671937,  0.70155902,  0.13454545,  0.141527  ,
         0.01288245,  0.67135135,  0.        ,  0.        ],
       [ 0.54112337,  0.59288538,  0.85523385,  0.15636364,  0.36312849,
         0.0273752 ,  0.46594595,  0.        ,  0.        ],
       [ 0.32748245,  0.38537549,  0.81737194,  0.31636364,  0.61452514,
         0.09822866,  0.35351351,  0.        ,  0.        ],
       [ 0.23119358,  0.42094862,  0.78396437,  0.66545455,  0.53072626,
         0.11111111,  0.27459459,  0.        ,  0.        ],
       [ 0.36108325,  0.48814229,  0.80846325,  0.28363636,  0.56238361,
         0.09178744,  0.32216216,  0.02857143,  0.45945946],
       [ 0.26128385,  0.55928854,  0.79510022,  0.42909091,  0.49162011,
         0.10305958,  0.27351351,  0.        ,  0.        ],
       [ 0.27883651,  0.49407115,  0.78841871,  0.43272727,  0

[[1.64846982 1.500371   2.97852859 1.81108068 1.45369407 0.50532314
  1.68423516 1.5932808  2.5976896 ]]
Testing time (k=1): 0.03 s.


AttributeError: 'NoneType' object has no attribute 'copy'

In [ ]:
train_labels = glass_train_df["CLASS"]
predictions = knn_model.predict(glass_train_df,k=1)
print("Accuracy on training set (k=1): {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set (k=1): {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set (k=1): {0:.4f}".format(brier_score(predictions,train_labels)))

### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class NaiveBayes

In [6]:
# Define the class NaiveBayes with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, binning, labels, class_priors, feature_class_value_counts, feature_class_counts
#
# Input to fit:
# self    - the object itself
# df      - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins  - no. of bins (default = 10)
# bintype - either "equal-width" (default) or "equal-size" 
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter              - a column filter (see Assignment 1) from df
# self.binning                    - a discretization mapping (see Assignment 1) from df
# self.class_priors               - a mapping (dictionary) from the labels (categories) of the "CLASS" column of df,
#                                   to the relative frequencies of the labels
# self.labels                     - a list of the categories (class labels) of the "CLASS" column of df
# self.feature_class_value_counts - a mapping from the feature (column name) to the number of
#                                   training instances with a specific combination of (non-missing, categorical) 
#                                   value for the feature and class label
# self.feature_class_counts       - a mapping from the feature (column name) to the number of
#                                   training instances with a specific class label and some (non-missing, categorical) 
#                                   value for the feature
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the naive approximation of Bayes rule (see lecture slides)
#
# Hint 1: First apply the column filter and discretization
#
# Hint 2: Iterating over either columns or rows, and for each possible class label, calculate the relative
#         frequency of the observed feature value given the class (using feature_class_value_counts and 
#         feature_class_counts) 
#
# Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
#         product of the relative frequencies
#
# Hint 4: Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors



In [7]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                       columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time (3, 'equal-width'): 0.10 s.
Testing time (3, 'equal-width'): 0.09 s.
Training time (3, 'equal-size'): 0.09 s.
Testing time (3, 'equal-size'): 0.07 s.
Training time (5, 'equal-width'): 0.10 s.
Testing time (5, 'equal-width'): 0.08 s.
Training time (5, 'equal-size'): 0.08 s.
Testing time (5, 'equal-size'): 0.07 s.
Training time (10, 'equal-width'): 0.09 s.
Testing time (10, 'equal-width'): 0.07 s.
Training time (10, 'equal-size'): 0.10 s.
Testing time (10, 'equal-size'): 0.07 s.



'results'

Accuracy  Brier score       AUC
3  equal-width  0.616822     0.622116  0.724335
   equal-size   0.607477     0.554782  0.780163
5  equal-width  0.644860     0.551101  0.771688
   equal-size   0.598131     0.581556  0.796675
10 equal-width  0.654206     0.527569  0.812887
   equal-size   0.588785     0.741668  0.751165

In [8]:
train_labels = glass_train_df["CLASS"]
nb_model.fit(glass_train_df)
predictions = nb_model.predict(glass_train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.8505
AUC on training set: 0.9687
Brier score on training set: 0.2263


### Comment on assumptions, things that do not work properly, etc.